<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/YOLOv8-eyecrop_Super%20extend%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv8-eye_crop_Super extend dataset**

YOLO5の眼周囲切り抜き精度アップのため、olympia+handai でトレーニングする (MobileNetは割愛)

```
Olympia+handai dataset
Dlibで目が2つ検出されるものを抜き出す
YOLOv8を用いて左右とバウンディングボックスを認識させる
```

###はじめの設定

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import pandas as pd
import csv
from random import randint
from time import sleep
import numpy as np
import sys
import torch
from tqdm import tqdm
import cv2
#from google.colab.patches import cv2_imshow
import seaborn as sns


import glob
import random
from PIL import Image
%matplotlib inline

# #サポートパッチのインポート
# from google.colab.patches import cv2_imshow
# import cv2

plt.ion()   # interactive mode
!nvidia-smi
print(torch.cuda.is_available())

from google.colab import drive
drive.mount('/content/drive')


Tue Apr  2 09:56:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
orig_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5/dataset_orig"

dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/uni_broad_for_YOLO8/"

# dstフォルダが存在する場合は削除する
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

# 新しいフォルダ構造を作成する
os.makedirs(dst_dir)
os.makedirs(os.path.join(dst_dir, "temp", "images"))
os.makedirs(os.path.join(dst_dir, "temp", "labels"))
os.makedirs(os.path.join(dst_dir, "images", "train"))
os.makedirs(os.path.join(dst_dir, "images", "val"))
os.makedirs(os.path.join(dst_dir, "labels", "train"))
os.makedirs(os.path.join(dst_dir, "labels", "val"))

# classes.txtファイルを作成する
with open(os.path.join(dst_dir, "classes.txt"), "w") as f:
    pass

#**HaarCascadeを用いて目を検出**

In [7]:
import cv2
import os

# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

orig_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5/dataset_orig"
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/uni_broad_for_YOLO8/"

out_images_dir = f"{dst_dir}/temp/images"
out_labels_dir = f"{dst_dir}/temp/labels"

# 出力ディレクトリが存在しない場合は作成
os.makedirs(out_images_dir, exist_ok=True)
os.makedirs(out_labels_dir, exist_ok=True)

files = os.listdir(orig_dir)

for file in files:  # フォルダ数の分だけ
    file_path = f"{orig_dir}/{file}"
    id = os.path.splitext(os.path.basename(file_path))[0]
    img = cv2.imread(file_path)
    img2 = img.copy()

    # 画像グレースケール化
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 300pix以上のもので目に見えるものを抽出
    eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))

    # 眼検出判定
    if len(eye_list) >= 1:
        print('目が' + str(len(eye_list)) + '個検出されました')
    else:
        print("eye detection error")

    # 画像の切り抜きと保存（2個以上検出の時に限る）
    if len(eye_list) >= 2:
        for (ex, ey, ew, eh) in eye_list:
            print(f"img_width: {img2.shape[1]}")
            print("[ex,ey] = %d,%d [ew,eh] = %d,%d" % (ex, ey, ew, eh))
            cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

            # Check if coordinates are within the image bounds
            ey_start = max(int(ey - 0.3 * eh), 0)
            ey_end = min(int(ey + 1.3 * eh), img.shape[0])
            ex_start = max(int(ex - 0.3 * ew), 0)
            ex_end = min(int(ex + 1.3 * ew), img.shape[1])

            # Ensure we have a valid crop area
            if ex_start < ex_end and ey_start < ey_end:
                img_cropped = img[ey_start: ey_end, ex_start: ex_end]

                if ex + eh * 0.5 <= img2.shape[1] / 2:
                    side = "R"  # 横幅の半分より左にあるのは右眼
                else:
                    side = "L"  # 横幅の半分よりより右にあるのは左眼

                print(f"side: {side}")
                print("")

                cv2.imwrite(f"{out_images_dir}/{id}_{side}.png", img_cropped)

                # YOLOのラベルを記載したtextファイルを作成
                with open(f"{out_labels_dir}/{id}_{side}.txt", "w") as f:
                    if side == "R":
                        label = 0
                    else:
                        label = 1

                    # YOLO座標の計算
                    yolo_x = (ex_start + ex_end) / (2 * img.shape[1])
                    yolo_y = (ey_start + ey_end) / (2 * img.shape[0])
                    yolo_w = (ex_end - ex_start) / img.shape[1]
                    yolo_h = (ey_end - ey_start) / img.shape[0]

                    f.write(f"{label} {yolo_x:.6f} {yolo_y:.6f} {yolo_w:.6f} {yolo_h:.6f}")
            else:
                print("Invalid crop coordinates. Skipping.")
    else:
        print("Less than 2 eyes detected. Skipping.")

ストリーミング出力は最後の 5000 行に切り捨てられました。

img_width: 2784
[ex,ey] = 359,903 [ew,eh] = 537,537
side: R

img_width: 2784
[ex,ey] = 1745,599 [ew,eh] = 909,909
side: L

img_width: 2784
[ex,ey] = 148,631 [ew,eh] = 1099,1099
side: R

目が2個検出されました
img_width: 2992
[ex,ey] = 287,601 [ew,eh] = 727,727
side: R

img_width: 2992
[ex,ey] = 2185,581 [ew,eh] = 699,699
side: L

目が2個検出されました
img_width: 2304
[ex,ey] = 316,609 [ew,eh] = 512,512
side: R

img_width: 2304
[ex,ey] = 1541,667 [ew,eh] = 444,444
side: L

目が2個検出されました
img_width: 2304
[ex,ey] = 1391,617 [ew,eh] = 686,686
side: L

img_width: 2304
[ex,ey] = 161,541 [ew,eh] = 713,713
side: R

目が2個検出されました
img_width: 2992
[ex,ey] = 226,799 [ew,eh] = 760,760
side: R

img_width: 2992
[ex,ey] = 2038,764 [ew,eh] = 825,825
side: L

目が2個検出されました
img_width: 2992
[ex,ey] = 493,889 [ew,eh] = 705,705
side: R

img_width: 2992
[ex,ey] = 1905,839 [ew,eh] = 754,754
side: L

目が2個検出されました
img_width: 2000
[ex,ey] = 154,334 [ew,eh] = 631,631
side: R

img_width: 2000
[ex,ey] = 1219,35

KeyboardInterrupt: 

In [6]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

#**目が2つ以上検出されたものを抜き出す**

dlibで検出されたものから、上下左右に0.1倍ずつ拡大した範囲を抜き出している

In [ ]:
with open(csv_coordinate_path, 'w', newline='') as f:
        #fieldnames = ['Number', 'Folder', 'FileName']
        writer = csv.writer(f)
        writer.writerow(['id','img_path', 'side R/L', 'ex', 'ey', 'ew', 'eh'])  #header

        files = os.listdir(orig_dir)

        k=0
        for file in files:  #フォルダ数の分だけ
              file_path = f"{orig_dir}/{file}"
              id = os.path.splitext(os.path.basename(file_path))[0]

              img = cv2.imread(file_path)
              img2 = img.copy()

              # 画像グレースケール化
              grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

              #300pix以上のもので目に見えるものを抽出
              eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))

              # 眼検出判定
              if len(eye_list) >= 1:
                  print('目が' + str(len(eye_list)) +'個検出されました')
              else:
                  print("eye detection error")

              #画像の切り抜きと保存（2個以上検出の時に限る）
              if len(eye_list) >= 2:
                  for (ex, ey, ew, eh) in eye_list:
                      print(f"img_width: {img2.shape[1]}")
                      print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))
                      cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                      img_cropped = img[int(ey-0.1*eh): int(ey+1.1*eh), int(ex-0.1*ew): int(ex+1.1*ew)] #本来の切り抜きより幅の0.1倍ずつ水増しする
                      #img_cropped = img[int(ey): int(ey+eh), int(ex): int(ex+ew)]


                      if ex+eh*0.5 <= img2.shape[1]/2:
                          side = "R" #横幅の半分より左にあるのは右眼
                      else:
                          side = "L" #横幅の半分よりより右にあるのは左眼

                      print(f"side: {side}")
                      print("")

                      # Check if coordinates are within the image bounds
                      ey_start = max(int(ey - 0.1 * eh), 0)
                      ey_end = min(int(ey + 1.1 * eh), img.shape[0])
                      ex_start = max(int(ex - 0.1 * ew), 0)
                      ex_end = min(int(ex + 1.1 * ew), img.shape[1])

                      # Ensure we have a valid crop area
                      if ex_start < ex_end and ey_start < ey_end:
                          img_cropped = img[ey_start: ey_end, ex_start: ex_end]

                          # Now do the checks for the right/left side, write image and row
                          # ...

                          cv2.imwrite(f"{out_dir}/{id}_{side}.png", img_cropped)

                      #対応表の作成
                      writer.writerow([id, file_path, side, ex-round(ew*0.1), ey-round(eh*0.1), round(ew*1.2), round(eh*1.2)])

                      #cv2_imshow(img_cropped)
                  else:
                      pass


##**ここで、目以外が誤検出されているものを手動で抜き出して削除する**

```
coordinate_uni_for_YOLO5.csvから、削除して画像のパスが存在しなくなっている行を削除する

```

In [ ]:
import os
import pandas as pd

# DataFrameを読み込む
coordinates_df = pd.read_csv(csv_coordinate_path)

# 存在しない画像パスをチェックし、そのリストを保持する
nonexistent_paths = coordinates_df[~coordinates_df['img_path'].apply(os.path.exists)]

# 存在しない画像パスを表示
print("Nonexistent image paths:")
print(nonexistent_paths['img_path'])

# # 存在しない画像パスの行を削除
# coordinates_df = coordinates_df[coordinates_df['img_path'].apply(os.path.exists)]

# # 更新されたDataFrameを保存する
# coordinates_df.to_csv('coordinate_uni_for_YOLO5.csv', index=False)

In [ ]:
'''
画像パスの抽出（RLともに揃っているもの）
'''
coordinates_filtered_df = coordinates_df.drop_duplicates(subset='id', keep='first')
img_path_list = coordinates_df['img_path'].tolist()


'''
画像の分割 train:valid = 8:2
'''
random.seed(42)  # For reproducibility
random.shuffle(img_path_list)

split_index = int(0.8 * len(img_path_list))
train_img_paths = img_path_list[:split_index]
valid_img_paths = img_path_list[split_index:]

if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

'''
フォルダの作成
'''
folders = ['train/images', 'train/labels', 'valid/images', 'valid/labels']
for folder in folders:
    os.makedirs(os.path.join(out_dir, folder))


'''
画像のコピー
'''
# Define paths for images directories
train_images_dir = os.path.join(out_dir, 'train/images')
valid_images_dir = os.path.join(out_dir, 'valid/images')

# Copy training images
for img_path in tqdm(train_img_paths, desc='Copying train images'):
    shutil.copy(img_path, train_images_dir)

# Copy validation images
for img_path in tqdm(valid_img_paths, desc='Copying valid images'):
    shutil.copy(img_path, valid_images_dir)

In [ ]:
# YOLO training folderへの振り分け
import os
import pandas as pd
from PIL import Image

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

def convert_to_yolo_format(ex, ey, ew, eh, img_width, img_height):
    cx = (ex + (ew / 2)) / img_width
    cy = (ey + (eh / 2)) / img_height
    w = ew / img_width
    h = eh / img_height
    return cx, cy, w, h

def create_label_files(image_dir, label_dir, df):
    image_files = os.listdir(image_dir)
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(image_dir, image_file)
        img_width, img_height = get_image_dimensions(image_path)

        base_name = os.path.splitext(image_file)[0]
        matched_rows = df[df['id'] == int(base_name)]

        if matched_rows.empty:
            raise ValueError(f"No matching id found for image {image_file}")

        label_file_path = os.path.join(label_dir, f"{base_name}.txt")
        with open(label_file_path, 'w') as label_file:
            for _, row in matched_rows.iterrows():
                ex = row['ex']
                ey = row['ey']
                ew = row['ew']
                eh = row['eh']
                cx, cy, w, h = convert_to_yolo_format(ex, ey, ew, eh, img_width, img_height)
                side = 0 if row['side R/L'] == 'R' else 1
                label_file.write(f" {side} {cx} {cy} {w} {h}\n")
                #label_file.write(f"{ex} {ey} {ew} {eh} {side}\n")

# CSVファイルの読み込み
csv_coordinate_df = pd.read_csv(csv_coordinate_path)

# trainとvalidのディレクトリパス
train_images_dir = os.path.join(out_dir, "train/images")
train_labels_dir = os.path.join(out_dir, "train/labels")
valid_images_dir = os.path.join(out_dir, "valid/images")
valid_labels_dir = os.path.join(out_dir, "valid/labels")

# trainディレクトリでラベルファイルを生成
create_label_files(train_images_dir, train_labels_dir, csv_coordinate_df)

# validディレクトリでラベルファイルを生成
create_label_files(valid_images_dir, valid_labels_dir, csv_coordinate_df)


In [ ]:
##############################
## バウンディングボックスのサンプル描画 ##
## (これは実行しなくて良い)            ##
##############################

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# バウンディングボックスを描画する関数
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

def draw_bounding_boxes(image_path, bboxes):
    with Image.open(image_path) as img:
        img_width, img_height = img.size
        fig, ax = plt.subplots(1)
        ax.imshow(img)
        for bbox in bboxes:
            class_id, cx, cy, bw, bh = bbox
            x = (cx - bw / 2) * img_width
            y = (cy - bh / 2) * img_height
            width = bw * img_width
            height = bh * img_height
            rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()

# ラベルファイルからバウンディングボックスのリストを取得する関数
def get_bboxes_from_label_file(label_path, img_width, img_height):
    bboxes = []
    with open(label_path, 'r') as file:
        for line in file:
            cx, cy, bw, bh, class_id = map(float, line.split())
            bboxes.append((cx, cy, bw, bh, class_id))
    return bboxes

# 画像パスとラベルファイルパス
image_path = f"{parent_dir}/dataset_uni_for_YOLOv5/dataset_uni/valid/images/10.JPG"
label_path = f"{parent_dir}/dataset_uni_for_YOLOv5/dataset_uni/valid/labels/10.txt"

# 画像のサイズを取得
img_width, img_height = get_image_dimensions(image_path)

# ラベルファイルからバウンディングボックスのリストを取得
bboxes = get_bboxes_from_label_file(label_path, img_width, img_height)


# バウンディングボックスを描画
print(f"img_width: {img_width}, img_height: {img_height}")
print(f"bboxes: {bboxes}")
draw_bounding_boxes(image_path, bboxes)


In [ ]:
%cd $out_dir

In [ ]:
%%writefile dataset.yaml
train: /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/train/images
val: /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv5/dataset_uni/valid/images

nc: 2
names: ['right', 'left']

##Train YOLOv5##

In [ ]:
# Train
!python train.py --img 640 --batch 16 --epochs 300 --data $out_dir/dataset.yaml --weights yolov5n.pt

In [ ]:
# 途中から
!python train.py --img 640 --batch 16 --epochs 300 --data $out_dir/data.yaml --resume $out_dir/yolov5/runs/train/exp/weights/last.pt

In [ ]:
# best.pyをrenameしてgdriveに移動しておく
orig_pt = f"{out_dir}/yolov5/runs/train/exp/weights/best.pt"
dst_pt = f"{out_dir}/eyecrop-yolov5n-300epoch.pt"
shutil.copy(orig_pt, dst_pt)

dst_pt = f"/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5_backup/eyecrop-yolov5n-300epoch.pt"
shutil.copy(orig_pt, dst_pt)

##YOLOv5 Inference original dataset


In [ ]:
%cd $out_dir/yolov5

import torch
import utils
%pip install -qr requirements.txt
display = utils.notebook_init()

In [ ]:
# YOLOv5
#weight = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/yolov5-iFish/runs/train/exp2/weights/best.pt"
#weight = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/yolov5n-iFish_120epoch.pt"
#weight = f"{out_dir}/dataset_uni/eyecrop-yolov5n-iFish_169epoch.pt"
shutil.copy("/content/drive/MyDrive/Deep_learning/GO_extended_dataset//dataset_uni_for_YOLOv5_backup/eyecrop-yolov5n-300epoch.pt", "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5/dataset_uni/eyecrop-yolov5n-300epoch.pt")
weight = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5/dataset_uni/eyecrop-yolov5n-300epoch.pt"

# もともとのデータセット
orig_dir = orig_dir #元画像
cropped_dir = cropped_dir #YOLOv5で切り抜いた画像用

# if os.path.exists(cropped_dir):
#     shutil.rmtree(cropped_dir)
# os.makedirs(cropped_dir)


In [ ]:
from models.common import DetectMultiBackend
#from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox #padding

from PIL import Image
import torch
from torchvision import models, transforms
import cv2
import numpy as np

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


def inference(img, weight):
    device = 'cpu'
    device = select_device(device)
    model = DetectMultiBackend(weight, device=device, dnn=False)
    #stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    #imgsz = check_img_size([640], s=stride)  # check image size

    #class_names = {0:"cont", 1:"grav"}

    # transform = transforms.Compose([
    #             transforms.Resize(size=(480,640)),
    #             transforms.ToTensor(),
    #             # transforms.Normalize(
    #             #     mean=[0.5, 0.5, 0.5],
    #             #     std=[0.5, 0.5, 0.5]
    #             #    )
    #             ])

    img_cv2 = cv2.imread(img) #CV2で開く
    img_cv2 = letterbox(img_cv2, (640,640), stride=32, auto=False)[0] #resize, 上下padding (color 114)

    #cv2_imshow(img_cv2)

    img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_cv2 = np.ascontiguousarray(img_cv2)
    img_tensor = torch.from_numpy(img_cv2).float()

    #img_tensor = transform(img_np)
    img_tensor /= 255
    #print(img_tensor.shape)

    #print(img_tensor)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


    pred = model(img_tensor, visualize=False, augment=False)

    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)

    print(f"pred: {pred}")

    return pred


def make_letterbox_image(cv2_image): #letterbox_image作成、リサイズ
    # 元の画像のサイズを取得
    height, width = cv2_image.shape[:2]

    # 正方形のサイズを決定（元の画像の長辺に合わせる）
    square_size = max(width, height)

    # 正方形のキャンバスを作成（背景は黒）
    letterbox_img = np.zeros((square_size, square_size, 3), dtype=np.uint8)

    # 元の画像を正方形の画像の中央に配置するための開始点（x,y）を計算
    x_center = (square_size - width) // 2
    y_center = (square_size - height) // 2

    # 元の画像を正方形のキャンバスにコピーする
    letterbox_img[y_center:y_center+height, x_center:x_center+width] = cv2_image

    letterbox_img_resized = cv2.resize(letterbox_img,(250,250))

    # 変換した画像を保存
    return letterbox_img_resized

In [ ]:
###############################
## バウンディングボックス&切り抜き demo ##
###############################
"""
Letterbox & 250px正方形にリサイズ
"""
import cv2
from google.colab.patches import cv2_imshow
import glob

image_path = glob.glob(f"{orig_dir}/*")
start_index = 1
end_index = 5

class_names = {0: "right", 1: "left"}

for i in range(start_index, end_index + 1):
    img = image_path[i]
    print(img)
    pred = inference(img, weight)  # Ensure this function is defined

    img_cv2 = cv2.imread(img)
    # Assume the inference function requires a certain image size; resize if needed.
    # Replace (640, 640) with the input size expected by your network.
    img_cv2_resized = cv2.resize(img_cv2, (640, int(img_cv2.shape[0] * 640 / img_cv2.shape[1])))  # 横幅を640pxにリサイズ

    for bbox in pred[0]:
        x1, y1, x2, y2, prob, class_num = torch.round(bbox[:6])

        prob = bbox[4].item()
        class_name = class_names[bbox[5].item()]

        print(f"診断は {class_name}、確率は{prob * 100:.1f}%です。")

        # calculate coordinates of the bounding box (640*640にpaddingされている分の座標を足す)
        img_height, img_width, _ = img_cv2_resized.shape[:3]
        print(f"img_height: {img_height}, img_width: {img_width}")
        padding_x = (img_height - min(img_width, img_height)) / 2
        padding_y = (img_width - min(img_width, img_height)) / 2
        x1 = x1 - padding_x
        y1 = y1 - padding_y
        x2 = x2 - padding_x
        y2 = y2 - padding_y
        print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
        if x1<0: #バウンディングボックスが左に切れる場合の対処
            x1 = 0
        if y2>img_width: #バウンディングボックスが右に切れる場合の対処
            x2 = img_width

        # Crop and resize logic (assuming make_letterbox_image is defined)
        mag = 640 / img_cv2.shape[1]
        cropped_img = img_cv2[int(y1/mag):int(y2/mag), int(x1/mag):int(x2/mag)]
        letterbox_img = make_letterbox_image(cropped_img)  # Ensure this function is defined

        letterbox_img = make_letterbox_image(cropped_img)
        cv2_imshow(letterbox_img)

    # After all boxes are drawn, show and save the final image
    cv2_imshow(img_cv2_resized)


In [ ]:
import cv2
import glob
import os
import torch

# Load image paths
image_paths = glob.glob(f"{orig_dir}/*")
start_index = 0
end_index = len(image_paths)

# Define class names
class_names = {0: "R", 1: "L"}

# List to hold images with incorrect detections
incorrect_detections = []

# Iterate over images
for i in range(start_index, end_index):
    img_path = image_paths[i]
    pred = inference(img_path, weight)  # Ensure this function is defined and weight is loaded

    img_cv2 = cv2.imread(img_path)
    img_cv2_resized = cv2.resize(img_cv2, (640, int(img_cv2.shape[0] * 640 / img_cv2.shape[1])))  # Resize width to 640px

    # Check if the detections are not equal to 2
    if len(pred[0]) != 2:
        incorrect_detections.append((img_path, len(pred[0])))
        continue  # Skip the rest of the loop and do not process this image

    for bbox in pred[0]:
        x1, y1, x2, y2, prob, class_num = torch.round(bbox[:6]).tolist()

        img_height, img_width, _ = img_cv2_resized.shape[:3]
        print(f"img_height: {img_height}, img_width: {img_width}")
        padding_x = (img_height - min(img_width, img_height)) / 2
        padding_y = (img_width - min(img_width, img_height)) / 2
        x1 = x1 - padding_x
        y1 = y1 - padding_y
        x2 = x2 - padding_x
        y2 = y2 - padding_y
        print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
        if x1<0: #バウンディングボックスが左に切れる場合の対処
            x1 = 0
        if y2>img_width: #バウンディングボックスが右に切れる場合の対処
            x2 = img_width

        # Crop and resize logic (assuming make_letterbox_image is defined)
        mag = 640 / img_cv2.shape[1]
        cropped_img = img_cv2[int(y1/mag):int(y2/mag), int(x1/mag):int(x2/mag)]
        letterbox_img = make_letterbox_image(cropped_img)  # Ensure this function is defined

        # Save the cropped image
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        class_name = class_names[class_num]
        cropped_img_path = os.path.join(f"{cropped_dir}", f"{base_name}_{class_name}.png")
        cv2.imwrite(cropped_img_path, letterbox_img)
        print(f"Successfully saved, image {i}: {cropped_img_path}")

# Output images with incorrect detections
print("Images with incorrect detections:")
for img_path, num_detections in incorrect_detections:
    print(f"{img_path} - Number of detections: {num_detections}")
incorrect_paths = [path for path, _ in incorrect_detections]

In [ ]:
###############################
## うまく左右眼を検出できなかった例の確認 ##
###############################
"""
Letterbox & 250px正方形にリサイズ
"""
import cv2
from google.colab.patches import cv2_imshow
import glob

image_path = ["/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5/dataset_orig/92.JPG"]
#image_path = incorrect_paths
start_index = 0
end_index = len(image_path)

class_names = {0: "right", 1: "left"}

for i in range(start_index, end_index + 1):
    img = image_path[i]
    pred = inference(img, weight)  # Ensure this function is defined

    img_cv2 = cv2.imread(img)
    # Assume the inference function requires a certain image size; resize if needed.
    # Replace (640, 640) with the input size expected by your network.
    img_cv2_resized = cv2.resize(img_cv2, (640, int(img_cv2.shape[0] * 640 / img_cv2.shape[1])))  # 横幅を640pxにリサイズ

    for bbox in pred[0]:
        x1, y1, x2, y2, prob, class_num = torch.round(bbox[:6])

        prob = bbox[4].item()
        class_name = class_names[bbox[5].item()]

        print(f"診断は {class_name}、確率は{prob * 100:.1f}%です。")

        # calculate coordinates of the bounding box (640*640にpaddingされている分の座標を足す)
        img_height, img_width, _ = img_cv2_resized.shape[:3]
        print(f"img_height: {img_height}, img_width: {img_width}")
        padding_x = (img_height - min(img_width, img_height)) / 2
        padding_y = (img_width - min(img_width, img_height)) / 2
        x1 = x1 - padding_x
        y1 = y1 - padding_y
        x2 = x2 - padding_x
        y2 = y2 - padding_y
        print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")

        # Draw bounding box
        cv2.rectangle(img_cv2_resized, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

        # Crop and save the image
        mag = 640 / img_cv2.shape[1]

        cropped_img = img_cv2[int(y1/mag):int(y2/mag), int(x1/mag):int(x2/mag)] #バウンディングボックスで切り抜き
        letterbox_img = make_letterbox_image(cropped_img)
        cv2_imshow(letterbox_img)

    # After all boxes are drawn, show and save the final image
    cv2_imshow(img_cv2_resized)


In [ ]:
import os
import random
import shutil
import cv2

def letterbox_image(image_path, output_path):
    # Load the image
    image = cv2.imread(image_path)

    # Calculate padding sizes
    h, w = image.shape[:2]
    max_side = max(h, w)
    top = bottom = (max_side - h) // 2
    left = right = (max_side - w) // 2

    # Add black padding to make the image square
    square_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    # Resize the image to 640x640
    resized_image = cv2.resize(square_image, (1080, 1080))

    # Save the processed image
    cv2.imwrite(output_path, resized_image)

def select_and_process_files(input_directory, output_directory, num_files=5):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List all files in the input directory
    all_files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]

    # Randomly select num_files files
    selected_files = random.sample(all_files, num_files)

    # Copy and process selected files
    for file in selected_files:
        input_path = os.path.join(input_directory, file)
        output_path = os.path.join(output_directory, file)
        shutil.copy(input_path, output_path)
        letterbox_image(output_path, output_path)

    return selected_files

# Example usage
input_directory = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/dataset_uni_for_YOLOv5/dataset_orig'
output_directory = '/content/letterbox'  # Replace with your desired output directory path
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)
os.makedirs(output_directory)
selected_files = select_and_process_files(input_directory, output_directory)
print("Selected and processed files:", selected_files)
